In [15]:
import load
from plot import plot3D

# from mygp import mygp

import numpy as np
import pandas as pd
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process import kernels
from sklearn.preprocessing import StandardScaler

import skopt 
from skopt.space import Real

from icecream import ic


### Wrapper to Gaussian Process Regressor
Includes arguments for kappa and xi

In [16]:
# Class mygp - wrapper for sklearn GaussianProcessRegressor
class mygp(object):
    # constructor
    def __init__(
        self,  
        gp=None,  
        **kwargs    
    ):
    
        if gp:
            self.gp = gp    
        else:
            self.gp = GaussianProcessRegressor()    



        self.ndims=1
        self.sample_X = np.array([])
        self.sample_y = np.array([])

        
        self.kwargs = kwargs
        self.kappa = kwargs.pop("kappa",0.0)
        self.acquisition_function = kwargs.pop("acquisition_function","LCB")    
        self.xi = kwargs.pop("xi", 0.01)     

    # print the current state of the object
    def prn(self, **kwargs):


        print(f"ndims = {self.ndims}. sample shape = {self.sample_X.shape}")        
        print(f"kappa={self.kappa}")
        print(f"xi={self.xi}")
        print(f"acquisition_function={self.acquisition_function}")    
        print(self.sample_X)
        print(self.sample_y)   

        return

    # initialize the sample data
    def sample_init(self,X,y):

        X_shape = X.shape
        X_len =  len(X_shape)

        
        if X_len == 1:
            self.ndims = 1
        elif X_len == 2:
            self.ndims = X_shape[1]
        else:         
            raise ValueError(f'sample_init() sample_X has incompatiple shape"{X_shape}" for the gaussian fit method.')
        
        self.sample_X = X.copy()
        self.sample_y = y.copy()


        self.gp.fit(self.sample_X, self.sample_y)

        sample_pred , sample_std = self.gp.predict(self.sample_X, return_std=True)

        return sample_pred
    
    # fit the model to the sample data
    def predict(self, grid_X):
                           
        X_len = len(grid_X.shape)

        if X_len == 1 and self.ndims == 1:
            pass      
        elif X_len ==2 and grid_X.shape[1] == self.ndims:       
            pass        
        else:
            raise ValueError(f'predict() grid_X has shape "{grid_X.shape}" dimensions. ndims = {self.ndims}')   

        grid_pred, grid_std = self.gp.predict(grid_X, return_std=True)

        return grid_pred, grid_std

    # wrapper to log marginal likelihood    
    def log_marginal_likelihood(self, theta=None,eval_gradient=False, clone_kernel=True):

        return self.gp.log_marginal_likelihood( theta=theta, eval_gradient=eval_gradient,clone_kernel=clone_kernel  )
    
    # debug function
    def get_params(self):
        ic(self.gp.get_params(deep=True))

    # add a sample point to the sample data, and 
    def sample_add(self, X1, y1):

        if len(X1) != self.ndims: 
            raise ValueError(f'sample_add(). Error sample_X1 must have exactly "{self.ndims}"')      

        # add point to array
        X = np.concatenate((self.sample_X, [X1]),axis=0 )
        y = np.concatenate((self.sample_y, [y1]),axis=0)     

        self.sample_init(X,y)

        return
    
    # wrapper to the skopt gp_minimize function
    def gp_minimize(self):

        def surrogate_function(inputs):
            # Predict using the Gaussian Process surrogate model
            y_pred, stdev = self.gp.predict(np.atleast_2d(inputs), return_std=True) 
            return y_pred[0]  



        my_domain = [Real(0.0, 1.0, name=f'num_{self.ndims}') for i in range(self.ndims)]    
        
        res = skopt.gp_minimize(
            func=surrogate_function,
            dimensions=my_domain,  # The search space
            # base_estimator= self.gp, 
            acq_func=self.acquisition_function,
            kappa= self.kappa,
            n_calls = 4,  # high to be on the safe side
            
            x0=[list(x) for x in self.sample_X], # Sample X values
            y0= self.sample_y,                   # y target 
            verbose=False                         # debug output ??
        )
        
        print(f"gp_minimize() res.x = {res.x}")
        print(f"gp_minimize() res.fun = {res.fun}")

        suggested = np.array(res.x_iters[-1])
        print("Suggested next inputs to try:", suggested)

        del res  

        return

    # wrapper to the skopt gp_minimize function
    def op_minimize(self,X, base_estimator  ):

        if X is None:
            X=np.array([])
         
        values = np.array([])
        my_domain = [Real(0.0, 1.0, name=f'num_{self.ndims}') for i in range(self.ndims)]    

             
        acq_func_kwargs = {"xi": self.xi, "kappa": self.kappa}      
          
        opt = skopt.Optimizer(
            my_domain,
            base_estimator= self.gp  if base_estimator==None else base_estimator,
            acq_func=self.acquisition_function,
            n_initial_points=1,   
            random_state=0,
            acq_func_kwargs=acq_func_kwargs,
            acq_optimizer='sampling' 
        )
        
        opt.tell(self.sample_X.tolist()   , self.sample_y.tolist(), fit=True)
        possibles = opt.ask()

        # remember opt.models[0] is the base_optimizer in opt        
        ic(opt.models[0])
        if len(X)>0:
            match self.acquisition_function:
                case "LCB":    
                    values = skopt.acquisition.gaussian_lcb(X, opt.models[0], kappa=self.kappa )    
                case "EI": 
                    values = skopt.acquisition.gaussian_ei(X, opt.models[0], y_opt=np.min(self.sample_y), xi=self.xi )
                case "PI":
                    values = skopt.acquisition.gaussian_pi(X, opt.models[0], y_opt=np.min(self.sample_y), xi=self.xi )    
                case _:
                    raise ValueError(f"op_minimize() unknown acquisition function {self.acquisition_function}") 


        
        # print(f"op_minimize(): value = {values}")         
        # print(f"opt.ask()={possibles}")

        del opt

        return values, possibles
  




### Main Calculation loop for each function
 returns an X point

In [17]:


def calc(idx, add):
      
    sample_X, sample_y = load.load_function(idx,add, True)
    X_plot = None

    match idx:
          
        case 1:
        
            kernel = kernels.Matern(nu=2.5, length_scale=0.01 )
            g = GaussianProcessRegressor(kernel=kernel,n_restarts_optimizer=10)
            gp = mygp(g, kappa=0.1, acquisition_function="LCB", xi=0.01 )
            gp.sample_init(sample_X,sample_y)
            values, possibles = gp.op_minimize(X_plot, None)

        case 2:
            kernel = kernels.Matern(nu=2.5,length_scale=0.01)
            g = GaussianProcessRegressor(kernel=kernel,n_restarts_optimizer=10)
            gp= mygp(g, kappa=0.1, acquisition_function="LCB",xi=0.1 )
            gp.sample_init(sample_X,sample_y)
            values, possibles=gp.op_minimize(X_plot,"GP")

        case 3: 
            kernel = kernels.Matern(nu=2.5,length_scale=0.01)
            g = GaussianProcessRegressor(kernel=kernel,n_restarts_optimizer=10)
            gp = mygp(g, kappa=0.1, acquisition_function="EI", xi=0.1 )
            gp.sample_init(sample_X,sample_y)
            values, possibles =gp.op_minimize(X_plot, None)

        case 4:
            kernel = kernels.Matern(nu=2.5,length_scale=0.01)
            g = GaussianProcessRegressor(kernel=kernel,n_restarts_optimizer=10)
            gp = mygp(g, kappa=1.0, acquisition_function="EI", xi=0.1)
            gp.sample_init(sample_X,sample_y)
            values, possibles = gp.op_minimize(X_plot, None)          
            
        case 5:
            kernel = kernels.Matern(nu=2.5,length_scale=0.01)
            g = GaussianProcessRegressor(kernel=kernel,n_restarts_optimizer=10)
            gp = mygp(g, kappa=0.1, acquisition_function="LCB", xi=0.1)
            gp.sample_init(sample_X,sample_y)
            values, possibles = gp.op_minimize(X_plot, None)
            
        case 6:
            kernel = kernels.Matern(nu=2.5,length_scale=0.01)
            g = GaussianProcessRegressor(kernel=kernel,n_restarts_optimizer=10)
            gp= mygp(g, kappa=0.1, acquisition_function="LCB", xi=0.1)
            gp.sample_init(sample_X,sample_y)
            values, possibles = gp.op_minimize(X_plot, None)
        case 7:
            kernel = kernels.Matern(nu=2.5,length_scale=0.01)
            g = GaussianProcessRegressor(kernel=kernel,n_restarts_optimizer=10)
            gp = mygp(g, kappa=0.1, acquisition_function="LCB", xi=0.1)
            gp.sample_init(sample_X,sample_y)
            values, possibles = gp.op_minimize(None, "GP")
        case 8: 
            # sample_y *= -1.0
            kernel = kernels.Matern(nu=2.5,length_scale=0.01)
            g = GaussianProcessRegressor(kernel=kernel,n_restarts_optimizer=10)
            gp = mygp(g, kappa=1.0, acquisition_function="EI", xi=0.1)
            gp.sample_init(sample_X,sample_y)
            values, possibles = gp.op_minimize(None, None)

    ic(possibles)    
    if ic.enabled:
        gp.prn()
        
    return possibles


### **Main Loop**
prepares points in a suitable format for Capstone submission.

In [18]:

week_nbr=15
ic.disable()

possibles_list=[]

for idx in range(1,9):
    print(f"idx={idx}")
    possibles=calc(idx,week_nbr)
    possibles_list.append(possibles)



load.print_for_capstone (possibles_list)




idx=1
idx=2


/home/hmb/anaconda3/envs/ts2/lib/python3.10/site-packages/sklearn/gaussian_process/_gpr.py:660: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL: .

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)


idx=3
idx=4
idx=5
idx=6


/home/hmb/anaconda3/envs/ts2/lib/python3.10/site-packages/sklearn/gaussian_process/_gpr.py:660: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL: .

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)


idx=7
idx=8
0.681972-0.355095-
0.199530-0.192559-
0.013688-0.053408-0.019874-
0.860470-0.996998-0.885160-0.955317-
0.001819-0.959575-0.030757-0.019916-
0.001819-0.959575-0.030757-0.019916-0.950110-
0.010226-0.274212-0.201202-0.760497-0.868145-0.311636-
0.802404-0.991959-0.848777-0.976442-0.737251-0.974855-0.834004-0.975097-


## Work in progress - test each function individually

In [18]:
ic.enable()
p = calc(1,12)

a,b = load.load_function(5,15,True)
# print(a)
# print(b)

ic(np.min(b), np.max(b), np.argmax(b))

# ic(a.shape, b.shape)
# ic(np.min(b), np.max(b))
ic(a,b)


ic| X.shape: (10, 2), Xf.shape: (20, 2)
/home/hmb/anaconda3/envs/ts2/lib/python3.10/site-packages/sklearn/gaussian_process/_gpr.py:660: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL: .

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
ic| opt.models[0]: GaussianProcessRegressor(kernel=Matern(length_scale=0.01, nu=2.5),
                                            n_restarts_optimizer=10)
ic| possibles: [0.7334704221276807, 0.547437369826271]
ic| X.shape: (20, 4), Xf.shape: (20, 4)
ic| np.min(b): 0.1129397953712203
    np.max(b): 2136.952983936292
    np.argmax(b): 30
ic| a: array([[0.19144708, 0.03819337, 0.60741781, 0.41458414],
              [0.75865295, 0.53651774, 0.65600038, 0.36034155],
              [0.43834987, 0.8043397 , 0.21024527, 0.15129482],
              [0.70605083, 0.53419196, 0.26424335, 0.48208755],
              [0.8

ndims = 2. sample shape = (40, 2)
kappa=0.1
xi=0.01
acquisition_function=LCB
[[3.19403889e-01 7.62959374e-01]
 [5.74329215e-01 8.79898105e-01]
 [7.31023631e-01 7.32999876e-01]
 [8.40353417e-01 2.64731614e-01]
 [6.50114060e-01 6.81526352e-01]
 [4.10437137e-01 1.47554299e-01]
 [3.12691157e-01 7.87227779e-02]
 [6.83418169e-01 8.61057464e-01]
 [8.25072518e-02 4.03487506e-01]
 [8.83889829e-01 5.82253974e-01]
 [9.00482207e-01 4.69966907e-01]
 [8.62435284e-01 4.35285602e-01]
 [9.31133254e-02 3.15207447e-01]
 [4.89754340e-01 1.68448474e-01]
 [9.01326853e-01 6.73664537e-01]
 [3.21017192e-01 6.34970606e-01]
 [3.20446588e-01 4.20306643e-01]
 [6.38617906e-01 7.38397229e-01]
 [5.96485775e-01 6.14079667e-01]
 [4.83667262e-01 3.19428352e-01]
 [9.62274946e-01 9.08318430e-01]
 [4.69375962e-01 5.53139406e-01]
 [6.93894423e-02 6.26719378e-01]
 [5.96408512e-01 8.16733668e-01]
 [1.40910900e-01 3.17933099e-01]
 [5.86534707e-01 2.07327548e-01]
 [6.15339247e-01 1.83209052e-01]
 [9.06870304e-01 7.61517465e-01]

 0.63961937, 0.41761768, 0.12260384],
              [0.11987923, 0.86254031, 0.64333133, 0.84980383],
              [0.12688467, 0.15342962, 0.77016219, 0.19051811],
              [0.94478251, 0.83050925, 0.83227839, 0.20292205],
              [0.05244809, 0.43614877, 0.37845743, 0.18802639],
              [0.06304804, 0.86636135, 0.85864857, 0.2085756 ],
              [0.98882078, 0.66382651, 0.36648076, 0.99624833],
              [0.7633217 , 0.62834667, 0.78222976, 0.81919558],
              [0.14275584, 0.20216247, 0.93864415, 0.22320333],
              [0.17997633, 0.81948394, 0.36434617, 0.88796618],
              [0.51645398, 0.39496365, 0.95182759, 0.69880194],
              [0.26844685, 0.49315648, 0.00685878, 0.76232741],
              [0.3981843 , 0.88166606, 0.1166999 , 0.84462821],
              [0.30584223, 0.93636618, 0.95401692, 0.88360492],
              [0.17995479, 0.69481431, 0.78705686, 0.11803764],
              [0.49675268, 0.42376709, 0.68348299, 0.30627614],
  

(array([[0.19144708, 0.03819337, 0.60741781, 0.41458414],
        [0.75865295, 0.53651774, 0.65600038, 0.36034155],
        [0.43834987, 0.8043397 , 0.21024527, 0.15129482],
        [0.70605083, 0.53419196, 0.26424335, 0.48208755],
        [0.83647799, 0.19360965, 0.6638927 , 0.78564888],
        [0.68343225, 0.11866264, 0.82904591, 0.56757661],
        [0.55362148, 0.66734998, 0.32380582, 0.81486975],
        [0.35235627, 0.32224153, 0.11697937, 0.47311252],
        [0.15378571, 0.72938169, 0.42259844, 0.44307417],
        [0.46344227, 0.63002451, 0.10790646, 0.9576439 ],
        [0.67749115, 0.35850951, 0.47959222, 0.07288048],
        [0.58397341, 0.14724265, 0.34809746, 0.42861465],
        [0.30688872, 0.31687813, 0.62263448, 0.09539906],
        [0.51114177, 0.817957  , 0.72871042, 0.11235362],
        [0.43893338, 0.77409176, 0.37816709, 0.93369621],
        [0.22418902, 0.84648049, 0.87948418, 0.87851568],
        [0.72526172, 0.47987049, 0.08894684, 0.75976022],
        [0.355

In [21]:
# Finds the min/max from  the function data, so we can see how we are doing
# A min/max for initial points data 
# B nin/max for second block of points data 
# C min/max from query data
load.min_max_check(20)

function_1
initial block min=-0.0036060626443634764, max=7.710875114502849e-16 
final block min=-5.0398197759117167e-08, max=0.06591475853269306
user block min=-0.003762575048057693, max=0.0023697618151689473
arg max index = 18
function_2
initial block min=-0.06562362443733738, max=0.6112052157614438 
final block min=-0.02995426789481563, max=0.6948320857901906
user block min=-0.16127004008105472, max=0.6954167327128791
arg max index = 22
function_3
initial block min=-0.3989255131463011, max=-0.034835313350078584 
final block min=-0.14258556714413848, max=-0.031039389276529272
user block min=-0.4387220412527796, max=-0.0017902770546808788
arg max index = 33
function_4
initial block min=-22.108287785435817, max=-10.069633426012889 
final block min=-28.90327367383377, max=-12.758324216781741
user block min=-44.481239000682585, max=-3.754695676825303
arg max index = 61
function_5
initial block min=0.1129397953712203, max=258.3705254462536 
final block min=9.972331894593824, max=1088.85961